# MUON TOMOGRAPHY PRIMER
In this notebook I will showcase a couple of studies I made in order to show how muons can be used to investigate the compositions and strucutre of large chunks of material

I'm using Geant4 in order to simulate the interaction of particles with matter and doing the leap of faith to consider that "ground truth". Whatever limitations that might have down the line is out of the scope of this first study.

The first problem I will try to tackle is figuring out the effective density of a column of material using the flux of muons that manage to traverse it end to end. In order to do inference I will rely on the Bethe-Bloch equation: 
$$\begin{equation}
- \frac{dE}{dx} = \rho K z^2 \frac{Z}{A} \frac{1}{\beta^2} 
\left[ \frac{1}{2} \ln \left( \frac{2 m_e c^2 \beta^2 \gamma^2 T_{\text{max}}}{I^2} \right) - \beta^2 - \frac{\delta}{2} \right],
\end{equation}
$$
The last term $\delta/2$ can really get messy but fort this time I will consider it's assymptotic behaviour:

$$ \frac{\delta}{2} \approx \frac{1}{2} \left( \ln \left( \frac{\hbar \omega_p}{I} \right) + \ln(\gamma^2) - 1 \right),
$$

where $\omega_p$ depends on the density $\rho$. 
Which we will see is good enough for this first demostration. 

The next piece of modelling is the "range" or penetration of a muon in a given material which can be approximated with the so-called CSDA range: 

$$R(E) = \int_{0}^{E} \frac{1}{S(E')} \, dE'$$ 

(We will also see that this is indeed a rather good approximation to start with! )


This function is a bit annoying because it's not defined in closed form, but the good news is that it is monotonic so it can be inverted to obtain the minimum energy that a muon needs in order to penetrate a certain distance $x$ within a material: 

$$ \mathcal{E}(x) = R^{-1}(x)$$ 

In this type of settings calculating the muon momentum would be cool, but it's actually rather complicated/expensive. Good news is that knowing the distribution of momentum is easier and it is well studied. 

And we come the basic idea that we will put to test. The flux of muons that go through a given material can be written as:

$$ \mathcal{F} = \int_{E_\text{min}}^{\infty} f(E')\, dE'$$

where $E_\text{min}$ is obtained with our function $\mathcal{E}(x)$, for this we will need just to plug the material width we want to study!

Basically, given a measured flux $\mathcal{F}_\mathrm{exp}$ one can obtain infer the density $\rho$ of the material by inverting the $\mathcal{F}$ relationship. For that we will make use a well-known minimizer: Minuit.

In particular we will use as target a material produced of:
60% Standard rock as defined by the NIST, the rest is split between air and water, parametrising the "saturation" with $s$, such that: 
$$\rho_\mathrm{eff} = 60\% \  \rho_\mathrm{Rock} + 40 \% \left( s  \rho_\mathrm{H_2O}  + (1-s) \rho_\mathrm{Air}\right) $$

Provided we can measure the density of our pure rock and that we know the densities of water and air we will be able to learn how much water lies inside our material (or our ore). 

First we define our stopping power function

In [6]:
import numpy as np 
import scipy
from functools import partial

In [9]:
z_a = 0.5
n_A = 6.022e23
rho = 1.205e-3  # g/cm3
p = 3000
m = 105.7
g2 = (1.44e-15) ** 2
def stopping(p, m=105.7, I=1e-4, rho=1):
    E = np.sqrt(p**2 + m**2)
    n = n_A * z_a * rho  #
    gamma = E / m
    beta = p / E
    beta = np.sqrt(1 - 1 / gamma**2)
    K = 0.307
    # return 4*np.pi /0.511  * n/beta**2 * g2 * ( np.log(2 *0.511 * beta**2/(I*(1-beta**2))) - beta**2 )
    Tmax = (
        2 * 0.511 * beta**2 * gamma**2 / (1 + 2 * gamma * 0.511 / m + (0.511 / m) ** 2)
    )
    ret = K * z_a / beta**2
    ret *= (
        0.5 * np.log(2 * 0.511 * beta**2 * gamma**2 * Tmax / I**2)
        - beta**2
        - np.log(28.816 * np.sqrt(rho * z_a) * 1e-6 / I)
        - np.log(beta * gamma)
        + 1 / 2
        + 1 / 8 * Tmax**2 / (gamma * m) ** 2
    )
    return ret * rho
# MeV-1 m-3 MeV**2 m**2 = MeV/m
def inv_stopping(*args, **kwargs):
    return 1 / stopping(*args, **kwargs)

Right away we can calculate the penetration ranges within air and water:

let's find the range $R(E= 10 \ \mathrm{GeV};\text{Water})$


In [10]:
pmax = np.sqrt(10000**2 - 105.7**2)
rho = 1
csda = scipy.integrate.quad(
    partial(inv_stopping, m=105.7, I=1e-4, rho=rho), 10, pmax
)  # m
print("Water", csda[0], csda[0] * rho)  # g cm-2 


Water 4668.284889284072 4668.284889284072


We obtain the following value: 
 $R(E= 10 \ \mathrm{GeV};\text{Water}) = 4668 \ \mathrm{cm}$ or $4668 \ \mathrm{g \ cm}^{-2}$

 If we go to the PDG (https://pdg.lbl.gov/2014/AtomicNuclearProperties/MUE/muE_water_liquid.pdf) we see that the tabulated value is a bit smaller: 4279 cm. Here is maybe a good point to talk about the Elephan in the room. There's a lot of parameters other than the density that govern the passage of charged particles through matter. For this study we will keep the ration between the mass number and the atomic number constant to $Z/A = 0.5$ which I think is a good approximation for the type of materials we are studyin. Also, the ionisation energy is set to $I=100\ \mathrm{eV}$.  
 
 Let's see how far we can get with this.


Let'see the ranges of the air and rock:

In [12]:
rho = 1.205e-3  # kg/m3
csda = scipy.integrate.quad(
    partial(inv_stopping, m=105.7, I=1e-4, rho=rho), 10, pmax
)  # m
print("Air", csda[0] * rho)  # g cm-2
rho = 2.65  # kg/m3
csda = scipy.integrate.quad(
    partial(inv_stopping, m=105.7, I=1e-4, rho=rho), 10, pmax
)  # m
print("Pure Rock", csda[0] * rho)  # g cm-2


Air 3747.8282195820607
Pure Rock 4840.835213684107


One can check that the approximation for Rock is much better than air( https://pdg.lbl.gov/2014/AtomicNuclearProperties/MUE/muE_air_dry_1_atm.pdf) ,(https://pdg.lbl.gov/2014/AtomicNuclearProperties/MUE/muE_standard_rock.pdf). The first reason for this is that we are enforcing the asymptotic behaviour of the density effect. We will see though, that this won't stop of us from continuing. We want to check if first principles are enough! 

Let's assume our muons are beamed onto a cuboid target made of a material as described before, and the goal is to determine the water saturatino $s$. In this first simulation experiment we are beaming muons parallel to our defined $z$ axis onto one of the sides of our homogenous material piece. The width of the material is 10 meters. The energy distribution of these muons is kept simple, a flat distribution between $1 \ \mathrm{GeV}$ and $6 \ \mathrm{GeV}$. 

In [16]:
# Lets read in our muons

import pandas as pd

df = pd.read_csv("../input/muons1.txt", on_bad_lines="skip", header=0)
df = df.apply(pd.to_numeric, errors="coerce")
n_muons_gen = len(df)
# Let's figure out how many made it to the other side of the cube, for that we check how many are stopped and actually have a proper state at z = 7500 
df.eval("p1 = sqrt(px1**2 + py1**2 + pz1**2)", inplace=True)
df.eval("p0 = sqrt(px0**2 + py0**2 + pz0**2)", inplace=True)
df.query("p1 > 1 & p0 > 100", inplace=True)
df.query("z1 > 1000", inplace=True)
n_muons_pass = len(df)
eff = n_muons_pass/n_muons_gen
print(f"Flux = ({100*eff} +- {100* np.sqrt(eff*(1-eff)/n_muons_gen)} %")

Flux = (52.96999999999999 +- 0.2232119665250947 %


 Now it's time to figure out what is the composition of our material! For that we will off-hold to minuit the inference of the effective density of our material, together with the saturation of liquid. In my Geant4 simulation I had set a 60% of Rock and a 20% saturation for water. Let's see what do we obtain! (The code to do this generation can be found in src/)

  ![](../input/muon.png)

In [25]:
from iminuit import Minuit
from scipy.interpolate import CubicSpline

def distrib(x):
    if x < 1000 or x > 6000:
        return 0
    return 1 / 5000

def csda(rho):
    ps = np.linspace(1e3, 15e3, 100)  # range of momentum
    x = []
    for p in ps:
        x.append(
            scipy.integrate.quad(
                partial(inv_stopping, m=105.7, I=1e-4, rho=rho), 00, p
            )[0]
        )
    spl = CubicSpline(ps, x)
    inv_spl = CubicSpline(x, ps) 
    return spl,inv_spl

def flux(rho):
    _,inv_spl = csda(rho)
    return scipy.integrate.quad(distrib,inv_spl(1000),20e3)[0]

flux_ref = 0.5306
def objective(rho):
    return abs(flux(rho)-flux_ref)

m = Minuit(objective,rho=1)
m.limits["rho"] = (0.1, 4)
m.migrad()

print(m)
# Get the result
rho_opt = m.values["rho"]
flux_opt = flux(rho_opt)

print(f"Optimal rho: {rho_opt}")
print(f"flux({rho_opt}) = {flux_opt:.2f}")

rock_ref1 = 2.601 # This is the density value as measure with this same method by setting the saturation to 0!
s1 = ((rho_opt -0.6*rock_ref1)/0.4 - 1.205e-3)/(1-1.205e-3)
print(f"Saturation is: {s1:.2f}")

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.085e-05                  │              Nfcn = 48               │
│ EDM = 2.82e-05 (Goal: 0.0002)    │            time = 6.9 sec            │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬──────

 We obtain a final saturation value of 0.22 which indeed is not amazingly accurate but it definetely is close the actual value! 

 Let's wrap here. Hope this was an interesting journey demonstrating with a very simplistic model how it's possible to infer the inner structure of a large chunk of material using cosmic muons. Deviations from reality are numerous, starting with the angular and energy distributions of the incoming muons and following with the numerous simplifications we are assuming for the Bethe-Bloch equation.